## Publication: Continous Tomography Implementation
 This markdown is the implementation of the tomobase library for usage of comparing 3 sets of data 
 1. Binomial Decomposition
 2. GRS 
 3. Incremental
 

### Imports
 This section is used to import useful code for storing data

In [ ]:
from tomobase.log import logger
import numpy as np
import napari
from tomobase import data, phantoms, tiltschemes, processes
from tomobase.data import Sinogram, Volume
import pandas as pd
import os 


def get_directory():
    path = os.getcwd()
    path = os.path.join(path, '..\..\data')
    return path

def get_filename(*args):
    path = get_directory()
    path = os.path.join(get_directory(), *args)
    return path

numbers = [np.linspace(1, 65, 65),
           np.linspace(1, 33, 33),
           np.linspace(1, 129, 129)]

print(get_directory())           
vol = phantoms.nanocage()

### Generate TiltSeries

In [ ]:
viewer = napari.Viewer()
for i in range(len(numbers)):
    indices = numbers[i]
    scheme = tiltschemes.GRS(64.0, -64.0, 0)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'grs_%d.mrc' % len(indices)))
    value = ts._to_napari_layer(True)
    viewer._add_layer_from_data(*value)
    
    scheme = tiltschemes.Binary(64.0, -64.0, 8)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'bin_%d.mrc' % len(indices)))

    
    scheme = tiltschemes.Incremental(64.0, -64.0, 128/len(indices)-1)
    angles = np.array([scheme.get_angle() for j in indices])
    ts = processes.project(vol, angles)
    ts = processes.alignments.pad_sinogram(ts, 1024, 1024, inplace=False)
    ts.to_file(get_filename('tiltseries', 'inc_%d.mrc' % len(indices)))

    
    




In [ ]:
# for all files in the directory
viewer = napari.Viewer()
for filename in os.listdir(get_filename('tiltseries')):
    ts = Sinogram.from_file(get_filename('tiltseries', filename))
    ts = processes.alignments.translational_misalignment(ts, 0.2, 0.2)
    
value = ts._to_napari_layer(True)
viewer.add_layer(*value)

    

### Get Tilt Series 


In [ ]:
# for each tilt series pad till 1024 by 1024
import numpy as np

print("Original shapes:")
print("ts_grs.data.shape:", ts_grs.data.shape)
print("ts_bin.data.shape:", ts_bin.data.shape)

# Pad the first two dimensions to 1024 while keeping the third dimension unchanged
ts_grs.data = np.pad(ts_grs.data, ((0, 1024 - ts_grs.data.shape[0]), (0, 1024 - ts_grs.data.shape[1]), (0, 0)), 'constant')
ts_grs.data = np.pad(ts_bin.data, ((0, 1024 - ts_bin.data.shape[0]), (0, 1024 - ts_bin.data.shape[1]), (0, 0)), 'constant')


ts_grs_misaligned, offset_grs =  processes.alignments.translational_misalignment(ts_grs, 0.1, inplace=False, extend_return=True)
ts_binary_misaligned, offset_binary = processes.alignments.translational_misalignment(ts_bin, 0.1, inplace =False, extend_return=True)

In [4]:

ts_grs_aligned = processes.alignments.align_sinogram_xcorr(ts_grs_misaligned, inplace=False, extend_return=True)
ts_bin_aligned = processes.alignments.align_sinogram_xcorr(ts_binary_misaligned, inplace=False, extend_return = True)

In [ ]:
print(offset_binary)
print(ts_bin_aligned[1])

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.filters import threshold_otsu

name  = 'compare-nomask'
volume = volume_increment.data
vol = volume_grs

#threshold_value = threshold_otsu(volume.flatten())
#volume_masked = np.where(volume.data > threshold_value, volume, 0)
volume_masked = volume

psnr_value = psnr(vol.data, volume_masked, data_range =1.0)
ssim_value, _ = ssim(vol.data, volume_masked, full=True, multichannel=True, data_range =1.0)
if os.path.exists(os.path.join(save_path, 'psnr_ssims.csv')):
    df = pd.read_csv(os.path.join(save_path,'psnr_ssims.csv'))
else:
    df = pd.DataFrame()
data = {
    'Name': name,
    'PSNR': psnr_value,
    'SSIM': ssim_value
}
print('PSNR: {}'.format(psnr_value))
print('SSIM: {}'.format(ssim_value))

df2 = pd.DataFrame(data, index=[0])
df = pd.concat([df, df2])
df.to_csv(os.path.join(save_path, 'psnr_ssims.csv'), index=False)

